In [15]:
import os
os.chdir('../')

In [16]:
import nibabel as nb
import numpy as np

import trimeshpy
#from trimeshpy.trimesh_vtk import TriMesh_Vtk
from trimeshpy.trimeshflow_vtk import TriMeshFlow_Vtk
from trimeshpy.vtk_util import lines_to_vtk_polydata, save_polydata

import time

In [17]:
surface_file = '/Users/kristianeschenburg/Desktop/Research/Data/Surfaces/285345.L.acpc_dc.white.32k_fs_LR.surf.gii'
surface = nb.load(surface_file)

triangles = surface.darrays[1].data
vertices = surface.darrays[0].data

In [18]:
#mesh = TriMesh_Vtk(triangles=triangles,vertices=vertices)
tri_mesh_flow = TriMeshFlow_Vtk(triangles, vertices)

In [19]:
# Test parameters
nb_step = 10
diffusion_step = 5
saved_flow = trimeshpy.data.output_test_flow
saved_fib = trimeshpy.data.output_test_fib

# Test functions
start = time.time()
#points = tri_mesh_flow.laplacian_smooth(nb_step, diffusion_step, l2_dist_weighted=False, area_weighted=False, backward_step=False, flow_file=saved_flow)
#points = tri_mesh_flow.curvature_normal_smooth(nb_step, diffusion_step, area_weighted=True, backward_step=True, flow_file=saved_flow)
#points = tri_mesh_flow.positive_curvature_normal_smooth(nb_step, diffusion_step, area_weighted=True, backward_step=True, flow_file=saved_flow)
#points = tri_mesh_flow.mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
points = tri_mesh_flow.positive_mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
#points = tri_mesh_flow.volume_mass_stiffness_smooth(nb_step, diffusion_step, flow_file=saved_flow)
stop = time.time()
print (stop - start)

 step 10 on 10 done 
13.697108030319214


In [20]:
lines = np.memmap(saved_flow, dtype=np.float64, mode='r', shape=(nb_step, vertices.shape[0], vertices.shape[1]))
tri_mesh_flow.set_vertices_flow(np.array(lines))
tri_mesh_flow.display(display_name="Trimeshpy: Flow resulting surface")
tri_mesh_flow.display_vertices_flow(display_name="Trimeshpy: Flow visualization")

/Users/kristianeschenburg/anaconda3/lib/python3.6/site-packages/vtk/util/numpy_support.py:137: FutureWarning: Conversion of the second argument of issubdtype from `complex` to `np.complexfloating` is deprecated. In future, it will be treated as `np.complex128 == np.dtype(complex).type`.
  assert not numpy.issubdtype(z.dtype, complex), \


In [21]:
da = surface.darrays[0]

for l in np.arange(lines.shape[0]):
    
    coords = np.asarray(lines[l,:,:]).astype(np.float32)
    da.data = coords
    surface.darrays[0] = da
    
    fp = '/Users/kristianeschenburg/Desktop/Flow.L.{:}.surf.gii'.format(l+1)
    nb.gifti.write(filename=fp,image=surface)

/Users/kristianeschenburg/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: DeprecationWarning: giftiio.write function deprecated. Use nibabel.load() instead.

* deprecated from version: 2.1
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 4.0
  # Remove the CWD from sys.path while we load stuff.


In [24]:
np.save(file='/Users/kristianeschenburg/Desktop/TestSave.npz',arr=lines)

In [26]:
points.shape

(32492, 3)

In [27]:
tri_mesh_flow.positive_mass_stiffness_smooth?

Signature: tri_mesh_flow.positive_mass_stiffness_smooth(nb_iter=1, diffusion_step=1.0, flow_file=None, gaussian_threshold=0.2, angle_threshold=1.0)
Docstring: <no docstring>
File:      ~/Documents/Code/trimeshpy/trimeshpy/trimesh_class.py
Type:      method


In [37]:
tri_mesh_flow.get_vertices_flow()[1,10,1]

-19.598497724649324

In [38]:
lines[1,10,1]

-19.598497724649324

In [41]:
tri_mesh_flow.positive_mass_stiffness_smooth?

Signature: tri_mesh_flow.positive_mass_stiffness_smooth(nb_iter=1, diffusion_step=1.0, flow_file=None, gaussian_threshold=0.2, angle_threshold=1.0)
Docstring: <no docstring>
File:      ~/Documents/Code/trimeshpy/trimeshpy/trimesh_class.py
Type:      method
